# Boosting

In [6]:
library(tidyverse)

#install.packages('hablar')
#library(hablar)

#install.packages("tidyr")
#install.packages("rlang")
library(tidyr)

#install.packages('scales')
library(scales)

library(LICORS)

#install.packages("slider")
library(slider)

library(glmnet)

library(gbm)

In [7]:
sum_control_ATAC_bin_rm = read.csv("data/sum_control_normalized_ATAC_bin_rm.csv")

In [4]:
head(sum_control_ATAC_bin_rm)

X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,...,X692,X693,X694,X695,X696,X697,X698,X699,X700,ATAC_val
8.699646e-07,3.479859e-06,2.609894e-06,0.000000e+00,8.699646e-07,8.699646e-07,1.739929e-06,1.739929e-06,0.000000e+00,2.609894e-06,...,0,0,0,0,0,0,0,0,0.000000e+00,0.5931657
8.313727e-07,8.313727e-07,0.000000e+00,8.313727e-07,1.662745e-06,8.313727e-07,8.313727e-07,5.819609e-06,3.325491e-06,3.325491e-06,...,0,0,0,0,0,0,0,0,0.000000e+00,0.4089315
8.705774e-07,1.741155e-06,8.705774e-07,8.705774e-07,0.000000e+00,0.000000e+00,2.611732e-06,3.482309e-06,0.000000e+00,8.705774e-07,...,0,0,0,0,0,0,0,0,8.705774e-07,0.3058228
0.000000e+00,1.700685e-06,0.000000e+00,8.503423e-07,8.503423e-07,1.700685e-06,8.503423e-07,0.000000e+00,8.503423e-07,8.503423e-07,...,0,0,0,0,0,0,0,0,0.000000e+00,0.4764137
0.000000e+00,9.416577e-07,0.000000e+00,0.000000e+00,9.416577e-07,0.000000e+00,9.416577e-07,0.000000e+00,2.824973e-06,0.000000e+00,...,0,0,0,0,0,0,0,0,0.000000e+00,0.2862775
0.000000e+00,0.000000e+00,0.000000e+00,4.221123e-06,1.688449e-06,1.688449e-06,8.442245e-07,0.000000e+00,2.532674e-06,0.000000e+00,...,0,0,0,0,0,0,0,0,0.000000e+00,0.3169988


In [5]:
set.seed(0)

fit <- gbm(formula  = ATAC_val ~ ., 
               data = sum_control_ATAC_bin_rm, 
            n.trees =600, 
           cv.folds = 10)

Distribution not specified, assuming gaussian ...


ERROR: Error in serialize(data, node$con): error writing to connection


In [ ]:
fit

## Plot the training error and the CV RMSE

In [ ]:
pd <- tibble(rmse_cv = sqrt(fit$cv.error), 
             rmse_train = sqrt(fit$train.error)) %>%
  mutate(tree = row_number()) %>%
  pivot_longer(names_to = "key", values_to = "value", -tree)

ggplot(pd, aes(x=tree, y=value, color=key)) + 
  geom_line() + 
  geom_point() + 
  NULL

In [ ]:
gbm.perf(fit, method = "cv")

In [ ]:
fit$train.error[which.min(sqrt(fit$train.error))]

fit$cv.error[which.min(sqrt(fit$cv.error))]


In [ ]:
predictions <- fit$cv.fitted
observed <- as.data.frame(sum_control_ATAC_bin_rm$ATAC_val)

In [ ]:
combined <- cbind(predictions, observed)
colnames(combined) <- c("predictions", "observed")
head(combined)

In [ ]:
ggplot(data = combined, aes(x = observed, y = predictions)) + 
    geom_point(size = 0.5) + 
    geom_smooth(method = "lm", formula = y~x)

In [ ]:
cor(combined$observed, combined$predictions)
#500 trees: 0.68095964611291
#600 trees: 0.681618712436074

# Extract the predictors

In [ ]:
#summary(fit)

## Tuning the boosting

In [8]:
library(tidyverse)
library(randomForest)
library(caret)
#install.packages("ranger")
library(ranger)
#install.packages("e1071")
library(e1071)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin

Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift


Attaching package: ‘ranger’

The following object is masked from ‘package:randomForest’:

    importance



In [9]:
x <- getModelInfo(model = "glmnet")[[1]]
x$label
x$type
x$parameters

[1] "glmnet"

[1] "Regression"     "Classification"

parameter,class,label
alpha,numeric,Mixing Percentage
lambda,numeric,Regularization Parameter


In [10]:
training_x <- sum_control_ATAC_bin_rm %>% select(-ATAC_val)
training_y <- sum_control_ATAC_bin_rm %>% select(ATAC_val)

In [11]:
str(training_y)

'data.frame':	10303 obs. of  1 variable:
 $ ATAC_val: num  0.593 0.409 0.306 0.476 0.286 ...


In [12]:
# Random search with tune length 10
trControl <- trainControl(method = "repeatedcv", number = 5, repeats = 3, verboseIter = T)

fit <- train(x = training_x, 
             y = training_y$ATAC_val, 
        method = "gbm",
     trControl = trControl,
      tuneLength = 10
    )

In [ ]:
# Plot to check the hyperparameters 
plot(fit)

In [ ]:
#Finer grid (The values ARE NOT TUNED YET!)
trControl <- trainControl(method = "repeatedcv", number = 5, repeats = 3, verboseIter = T)
tuneGrid <- expand.grid(n.trees = c(100, 200, 250), interaction.depth = c(1, 4, 6), shrinkage = 0.1, n.minobsinnode = 10)
  
fit <- train(x = training_x, 
             y = training_y, 
        method = "gbm",
     trControl = trControl,
      tuneGrid = tuneGrid
    )

In [ ]:
# Plot to check the hyperparameters 
plot(fit)